# SymbolicAI: Basics

### Get Imports

In [6]:
import os
import warnings
warnings.filterwarnings('ignore')
os.chdir('../') # set the working directory to the root of the project
from symai import *
from IPython.display import display

### Using Symbols

In [2]:
sym = Symbol('Hi there!')
sym

<class 'symai.symbol.Symbol'>(value=Hi there!)

Adding two symbols together.

In [3]:
res = sym + 'how are you?'
res

<class 'symai.symbol.Symbol'>(value=Hi there! How are you?)

If one of the objects is a symbol then the other object is converted to a symbol.

A `Symbol` instance takes in any type of object and preserves its original value type until it is evaluated and sent to the neuro-symbolic engine:

In [4]:
sym = Symbol(np.array([5, 2, 42, 1]))
sym.type()

numpy.ndarray

One can also easily retriev the object by accessing the `value` attribute:

In [5]:
sym.value

array([ 5,  2, 42,  1])

We can add multiple values together and their operation will be contextualized evaluated. Here we use the shift operator to integrate a new value into the existing symbol:

In [6]:
res = sym << 2
res

<class 'symai.symbol.Symbol'>(value=['5', '2', '42', '1', '2'])

We could do the same manipulation with the `+` operator, however, the prompts designed for each individual operator may contain different information, therefore, therefore may not evaluated to similar outcomes. In the following example it is clear that the `+` operator is not to shift the value into the array:

In [7]:
res = sym + 2
res

<class 'symai.symbol.Symbol'>(value=['5', '2', '42', '1', 2])

However, if we use the `+` operator to evaluate arithmetic operations, the results are quite different:

In [8]:
x = Symbol('x')
x += 1
x

<class 'symai.symbol.Symbol'>(value=x1)

In that case the neural engines believes that the goal is to concatenate the two values together. Assuming that we want to add to mathematical operations, we need to use the `expression` operator:

In [10]:
sym = Symbol('x + 1').expression('self + 2')
sym

<class 'symai.symbol.Symbol'>(value=x + 3)

Here we get the expected result. The `self` statement in the `expression` method is a reference to the current symbol (x + 1). This is useful when we want to access values from the current symbol using the `expression` method.

### Evaluating Expressions

Similar to `word2vec` we intend to preform contextualized operations on different symbols. 

Word2vec is a machine learning algorithm that is used to generate dense vector representations of words. It works by training a shallow neural network to predict a word given its neighbors in a text corpus. The resulting vectors are then used in a wide range of natural language processing applications, such as sentiment analysis, text classification, and clustering.

Below we can see an example how one can perform operations on the word embedding vectors (colored boxes).

<img src="../assets/images/img3.png" width="470px">

However, we can also perform operations on the natural language space itself. For example, we can add two the word expressions similar to how we added them in the word2vec space:

In [3]:
Symbol('King - Man + Women').expression()

<class 'symai.symbol.Symbol'>(value=Queen)

As we can see, if the `expression` method is empty the default implementation of `Symbol` uses only the `value` stored within the current symbol. 

Now let's try to evaluate a more complex expression:

In [2]:
sym = Symbol()
res = sym.expression('x^2 + 2x + 1')
res


<class 'symai.symbol.Symbol'>(value=x^2 + 2x + 1)

Well, not exactly simplified the way we intended it. 

This is because using LLMs for expression evaluations is still very limited, therefore, to leverage the full power of symbolic programming we can use the `expression` method to evaluate an expression via `WolframAlpha`:

In [12]:
sym = Symbol()
Expression.command(engines=['symbolic'], expression_engine='wolframalpha')
res = sym.expression('x^2 + 2x + 1')
res

<class 'symai.symbol.Expression'>(value=x = -1)

There we go! The `command` method configures the `symbolic` engine to evaluate expressions via `WolframAlpha`. One can see that this evaluates the expression and returns the correct result.

### Showing other basic operations

Sometimes we want to simply concatenate to symbols together without any other neural operation. This is easily done with the `@` operation:

In [3]:
# define a second string
sym = Symbol("Welcome to our tutorial.")
test2 = 'Hello world!'
# concatenate strings
res = sym @ test2
res

<class 'symai.symbol.Symbol'>(value=Welcome to our tutorial.Hello world!)

We can of course define full sentences as Symbols and perform several operations on them.

In [2]:
sym = Symbol("""Images and videos of domestic cats make up some of the most viewed content on the World Wide Web, particularly image macros in the form of lolcats. ThoughtCatalog has described cats as the "unofficial mascot of the Internet".[1]
The subject has attracted the attention of various scholars and critics, who have analyzed why this form of low art has reached iconic status. Although it may be considered frivolous, cat-related Internet content contributes to how people interact with media and culture.[2] Some argue that there is a depth and complexity to this seemingly simple content, with a suggestion that the positive psychological effects that pets have on their owners also hold true for cat images viewed online.[3]
Research has suggested that viewing online cat media is related to positive emotions, and that it even may work as a form of digital therapy or stress relief for some users. Some elements of research also shows that feelings of guilt when postponing tasks can be reduced by viewing cat content.[4]
""")

Here we translate the existing Symbol to German:

In [3]:
sym.translate('German')

<class 'symai.symbol.Symbol'>(value=Bilder und Videos von Haustieren machen einen Großteil der am meisten angesehenen Inhalte im World Wide Web aus, insbesondere Bildmacros in Form von Lolcats. ThoughtCatalog beschrieb Katzen als den "unoffiziellen Maskottchen des Internets". [1]
Das Thema hat die Aufmerksamkeit verschiedener Wissenschaftler und Kritiker auf sich gezogen, die analysiert haben, warum diese Form der)

And now, we try to classify how the mood of the above sentences:

In [4]:
sym.choice(['angry', 'neutral', 'hate-speech', 'happy', 'unk'], default='unk')

<class 'symai.symbol.Symbol'>(value=neutral)

## Documentation Generator Example

In [15]:
from examples.docs import Docs, CppDocs
docs = Docs()

In [16]:
doc = docs("""def execute(default: str = None,
            constraints: List[Callable] = [],
            pre_processors: List[PreProcessor] = [],
            post_processors: List[PostProcessor] = [],
            *wrp_args,
            **wrp_kwargs):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(wrp_self, *args, **kwargs):
            return execute_func(wrp_self, 
                                func=func,
                                code=str(wrp_self),
                                constraints=constraints, 
                                default=default, 
                                pre_processors=pre_processors, 
                                post_processors=post_processors,
                                wrp_args=wrp_args,
                                wrp_kwargs=wrp_kwargs,
                                args=args, kwargs=kwargs)
        return wrapper
    return decorator
""")
print(doc)

"""Executes a function with the given parameters.

Args:
    default (str, optional): The default value to be returned if the task cannot be solved. Defaults to None.
    constraints (List[Callable], optional): A list of constrains applied to the model output to verify the output. Defaults to [].
    pre_processors (List[PreProcessor], optional): A list of pre-processors to be applied to the input and shape the input to the model. Defaults to None.
    post_processors (List[PostProcessor], optional): A list of post-processors to be applied to the model output and before returning the result. Defaults to [].
    *wrp_args: Variable length argument list.
    **wrp_kwargs: Arbitrary keyword arguments.

Returns:
    decorator: A decorator that wraps the given function with the given parameters.
"""


## Markdown

We can actually on the fly also generate markdown documentation for our notebooks. This is done by using the built in `query` method. The `query` method is used to query the `symbolic` engine for information about the current symbol. The `query` method can therefore be used to ask the engine to generate documentation in a specific format.

In [18]:
res = doc.query(context="Write a very detailed Markdown documentation the News class.")
print(res)

# News

The News class is a custom Expression class used to fetch and process news articles. It contains methods to fetch a news article from a given URL, process the article using a specified pattern, and render the article in an HTML template. 

## Constructor

The constructor takes four parameters:

- `url`: A string representing the URL of the news article to fetch
- `pattern`: A string representing the pattern to use when processing the news article
- `filters`: An optional list of Expressions used to filter the news article
- `render`: An optional boolean indicating whether or not to render the news article in an HTML template

The constructor also initializes the following fields:

- `data_stream`: A Stream object containing a Sequence of Clean, Translate, Outline, and Compose expressions
- `header_style`: A Style object used to style the HTML template
- `html_template`: A Symbol object containing the HTML template
- `html_template_seq`: A Template object used to insert the rend